In [1]:
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import geonamescache
from collections import Counter



In [2]:
#importing data

data = pd.read_csv("../CSV/ml_insurance_challenge.csv",encoding="ISO-8859-1")
#print(data.head())

pd.set_option('display.max_colwidth', None)

#data.head()

In [3]:
data.shape

(9494, 5)

In [4]:
#removing punctuation

string.punctuation

def remove_punctuation(text):
    punctuationfree = "".join([i for i in text if i not in string.punctuation])
    return punctuationfree

clean=data.copy()
clean['description'] = data['description'].apply(lambda x: remove_punctuation(str(x)))

clean['business_tags'] = data['business_tags'].apply(lambda x: remove_punctuation(str(x)))

clean['category'] = data['category'].apply(lambda x: remove_punctuation(str(x)))

clean['niche'] = data['niche'].apply(lambda x: remove_punctuation(str(x)))

clean.head()

,description,business_tags,sector,category,niche
0,Welchcivils is a civil engineering and construction company that specializes in designing and building utility network connections across the UK They offer multiutility solutions that combine electricity gas water and fibre optic installation into a single contract Their design engineer teams are capable of designing electricity water and gas networks from existing network connection points to meter locations at the development as well as project management of reinforcements and diversions They provide custom connection solutions that take into account any existing assets maximize the usage of every trench and meet project deadlines Welchcivils has considerable expertise installing gas and electricity connections in a variety of market categories including residential commercial and industrial projects as as well,Construction Services Multiutilities Utility Network Connections Design and Construction Water Connection Installation Multiutility Connections Fiber Optic Installation,Services,Civil Engineering Services,Other Heavy and Civil Engineering Construction
1,Kyoto Vegetable Specialists Uekamo also known as Iwamachi is a company based in Kyoto Japan that specializes in the sale of vegetables They have been in business for ten years and offer a collection of vegetable recipes through their KeiÅ Vegetable Recipe Collection and Online Shop The company is directly owned by Uekamoo Farm Uekame Farm and Lobechi Shijohara Farm They offer a variety of vegetable products including sugukizuke and Kamoo eggplant and also accept production cultivation according to customer requests Iwaichi Limited Company uses their experience in production and sales to provide tailored vegetables to meet customer needs and also accepts cultivation of products according to their requirements,Wholesale Dualtask Movement Products Cast Iron Products Manufacturer Manufacturing Technology Food and Beverage Rice And Noodles Highquality Gloss of Cast Iron Rice Wholesaler Miscellaneous Crop Farming Health and Wellness Products Agricultural Cooperative Medical Practice Based on Eastern Medicine Production Rice Pudding,Manufacturing,Fruit Vegetable Markets Stores,Frozen Fruit Juice and Vegetable Manufacturing
2,Loidholdhof Integrative Hofgemeinschaft is a company that offers a range of services and products to its customers Their products are all handmade and of the highest quality produced on a biodynamic basis with a focus on freshness and quality The companys product range includes homemade bread honey from their own beekeeping syrup and fresh vegetables which can be purchased in their farm shop In addition to their farm products they also have a farm shop and cafe where customers can enjoy fresh coffee and delicious cakes,Living Forms Farm Cafe Fresh Coffee Community Engagement Freshly Baked Bread Social Interaction Opportunities Fresh Vegetables Homemade Honey Delicious Cakes Communityoriented Living Handmade Products Fresh Juices Farm Fresh Products Integrated Farming Community Biodynamic Farming,Manufacturing,Farms Agriculture Production,All Other Miscellaneous Crop Farming
3,PATAGONIA Chapa Y Pintura is an auto body shop located in Comodoro Rivadavia Chubut Province Argentina The company specializes in providing auto body repair services,Automotive Body Repair Services Interior Repair Services,Services,Auto Body Shops,Automotive Body Paint and Interior Repair and Maintenance
4,Stanica WODNA PTTK Swornegacie is a cultural establishment located in Swornychgaciach Poland It is a popular destination for kayakers and tourists of all levels offering a variety of activities and events The establishment is managed by Zbigniew GaliÅski,Cultural Activities Accommodation Services Kayak Rentals Small Gastronomy Products Tourism Services Recreational Activities Cultural Center,Services,Boat Tours Cruises,Scenic and Sightseeing Transportation Water


In [5]:
clean = clean.applymap(lambda x: x.lower() if isinstance(x, str) else x)

clean.head()

C:\Users\anama_1lv\AppData\Local\Temp\ipykernel_15640\814097994.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clean = clean.applymap(lambda x: x.lower() if isinstance(x, str) else x)


,description,business_tags,sector,category,niche
0,welchcivils is a civil engineering and construction company that specializes in designing and building utility network connections across the uk they offer multiutility solutions that combine electricity gas water and fibre optic installation into a single contract their design engineer teams are capable of designing electricity water and gas networks from existing network connection points to meter locations at the development as well as project management of reinforcements and diversions they provide custom connection solutions that take into account any existing assets maximize the usage of every trench and meet project deadlines welchcivils has considerable expertise installing gas and electricity connections in a variety of market categories including residential commercial and industrial projects as as well,construction services multiutilities utility network connections design and construction water connection installation multiutility connections fiber optic installation,services,civil engineering services,other heavy and civil engineering construction
1,kyoto vegetable specialists uekamo also known as iwamachi is a company based in kyoto japan that specializes in the sale of vegetables they have been in business for ten years and offer a collection of vegetable recipes through their keiå vegetable recipe collection and online shop the company is directly owned by uekamoo farm uekame farm and lobechi shijohara farm they offer a variety of vegetable products including sugukizuke and kamoo eggplant and also accept production cultivation according to customer requests iwaichi limited company uses their experience in production and sales to provide tailored vegetables to meet customer needs and also accepts cultivation of products according to their requirements,wholesale dualtask movement products cast iron products manufacturer manufacturing technology food and beverage rice and noodles highquality gloss of cast iron rice wholesaler miscellaneous crop farming health and wellness products agricultural cooperative medical practice based on eastern medicine production rice pudding,manufacturing,fruit vegetable markets stores,frozen fruit juice and vegetable manufacturing
2,loidholdhof integrative hofgemeinschaft is a company that offers a range of services and products to its customers their products are all handmade and of the highest quality produced on a biodynamic basis with a focus on freshness and quality the companys product range includes homemade bread honey from their own beekeeping syrup and fresh vegetables which can be purchased in their farm shop in addition to their farm products they also have a farm shop and cafe where customers can enjoy fresh coffee and delicious cakes,living forms farm cafe fresh coffee community engagement freshly baked bread social interaction opportunities fresh vegetables homemade honey delicious cakes communityoriented living handmade products fresh juices farm fresh products integrated farming community biodynamic farming,manufacturing,farms agriculture production,all other miscellaneous crop farming
3,patagonia chapa y pintura is an auto body shop located in comodoro rivadavia chubut province argentina the company specializes in providing auto body repair services,automotive body repair services interior repair services,services,auto body shops,automotive body paint and interior repair and maintenance
4,stanica wodna pttk swornegacie is a cultural establishment located in swornychgaciach poland it is a popular destination for kayakers and tourists of all levels offering a variety of activities and events the establishment is managed by zbigniew galiåski,cultural activities accommodation services kayak rentals small gastronomy products tourism services recreational activities cultural center,services,boat tours cruises,scenic and sightseeing transportation water


In [6]:
#merging columns

# clean['merged'] = clean[['description', 'business_tags', 'sector', 'category', 'niche']].apply(lambda x: ' '.join(x.dropna()), axis=1)

# clean = clean['merged']

In [7]:
#removing stopwords

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return [word for word in text.split() if word not in stop_words]

clean['description'] = clean['description'].apply(lambda x: ' '.join(remove_stopwords(x)))
clean['business_tags'] = clean['business_tags'].apply(lambda x: ' '.join(remove_stopwords(x)))
clean['niche'] = clean['niche'].apply(lambda x: ' '.join(remove_stopwords(x)))

clean.head()

,description,business_tags,sector,category,niche
0,welchcivils civil engineering construction company specializes designing building utility network connections across uk offer multiutility solutions combine electricity gas water fibre optic installation single contract design engineer teams capable designing electricity water gas networks existing network connection points meter locations development well project management reinforcements diversions provide custom connection solutions take account existing assets maximize usage every trench meet project deadlines welchcivils considerable expertise installing gas electricity connections variety market categories including residential commercial industrial projects well,construction services multiutilities utility network connections design construction water connection installation multiutility connections fiber optic installation,services,civil engineering services,heavy civil engineering construction
1,kyoto vegetable specialists uekamo also known iwamachi company based kyoto japan specializes sale vegetables business ten years offer collection vegetable recipes keiå vegetable recipe collection online shop company directly owned uekamoo farm uekame farm lobechi shijohara farm offer variety vegetable products including sugukizuke kamoo eggplant also accept production cultivation according customer requests iwaichi limited company uses experience production sales provide tailored vegetables meet customer needs also accepts cultivation products according requirements,wholesale dualtask movement products cast iron products manufacturer manufacturing technology food beverage rice noodles highquality gloss cast iron rice wholesaler miscellaneous crop farming health wellness products agricultural cooperative medical practice based eastern medicine production rice pudding,manufacturing,fruit vegetable markets stores,frozen fruit juice vegetable manufacturing
2,loidholdhof integrative hofgemeinschaft company offers range services products customers products handmade highest quality produced biodynamic basis focus freshness quality companys product range includes homemade bread honey beekeeping syrup fresh vegetables purchased farm shop addition farm products also farm shop cafe customers enjoy fresh coffee delicious cakes,living forms farm cafe fresh coffee community engagement freshly baked bread social interaction opportunities fresh vegetables homemade honey delicious cakes communityoriented living handmade products fresh juices farm fresh products integrated farming community biodynamic farming,manufacturing,farms agriculture production,miscellaneous crop farming
3,patagonia chapa pintura auto body shop located comodoro rivadavia chubut province argentina company specializes providing auto body repair services,automotive body repair services interior repair services,services,auto body shops,automotive body paint interior repair maintenance
4,stanica wodna pttk swornegacie cultural establishment located swornychgaciach poland popular destination kayakers tourists levels offering variety activities events establishment managed zbigniew galiåski,cultural activities accommodation services kayak rentals small gastronomy products tourism services recreational activities cultural center,services,boat tours cruises,scenic sightseeing transportation water


In [8]:
clean['description'] = clean['description'].apply(lambda x: x.split() if isinstance(x, str) else x)

clean['business_tags'] = clean['business_tags'].apply(lambda x: x.split() if isinstance(x, str) else x)

clean['sector'] = clean['sector'].apply(lambda x: x.split() if isinstance(x, str) else x)

clean['category'] = clean['category'].apply(lambda x: x.split() if isinstance(x, str) else x)

clean['niche'] = clean['niche'].apply(lambda x: x.split() if isinstance(x, str) else x)

clean.head()    

,description,business_tags,sector,category,niche
0,"[welchcivils, civil, engineering, construction, company, specializes, designing, building, utility, network, connections, across, uk, offer, multiutility, solutions, combine, electricity, gas, water, fibre, optic, installation, single, contract, design, engineer, teams, capable, designing, electricity, water, gas, networks, existing, network, connection, points, meter, locations, development, well, project, management, reinforcements, diversions, provide, custom, connection, solutions, take, account, existing, assets, maximize, usage, every, trench, meet, project, deadlines, welchcivils, considerable, expertise, installing, gas, electricity, connections, variety, market, categories, including, residential, commercial, industrial, projects, well]","[construction, services, multiutilities, utility, network, connections, design, construction, water, connection, installation, multiutility, connections, fiber, optic, installation]",[services],"[civil, engineering, services]","[heavy, civil, engineering, construction]"
1,"[kyoto, vegetable, specialists, uekamo, also, known, iwamachi, company, based, kyoto, japan, specializes, sale, vegetables, business, ten, years, offer, collection, vegetable, recipes, keiå, vegetable, recipe, collection, online, shop, company, directly, owned, uekamoo, farm, uekame, farm, lobechi, shijohara, farm, offer, variety, vegetable, products, including, sugukizuke, kamoo, eggplant, also, accept, production, cultivation, according, customer, requests, iwaichi, limited, company, uses, experience, production, sales, provide, tailored, vegetables, meet, customer, needs, also, accepts, cultivation, products, according, requirements]","[wholesale, dualtask, movement, products, cast, iron, products, manufacturer, manufacturing, technology, food, beverage, rice, noodles, highquality, gloss, cast, iron, rice, wholesaler, miscellaneous, crop, farming, health, wellness, products, agricultural, cooperative, medical, practice, based, eastern, medicine, production, rice, pudding]",[manufacturing],"[fruit, vegetable, markets, stores]","[frozen, fruit, juice, vegetable, manufacturing]"
2,"[loidholdhof, integrative, hofgemeinschaft, company, offers, range, services, products, customers, products, handmade, highest, quality, produced, biodynamic, basis, focus, freshness, quality, companys, product, range, includes, homemade, bread, honey, beekeeping, syrup, fresh, vegetables, purchased, farm, shop, addition, farm, products, also, farm, shop, cafe, customers, enjoy, fresh, coffee, delicious, cakes]","[living, forms, farm, cafe, fresh, coffee, community, engagement, freshly, baked, bread, social, interaction, opportunities, fresh, vegetables, homemade, honey, delicious, cakes, communityoriented, living, handmade, products, fresh, juices, farm, fresh, products, integrated, farming, community, biodynamic, farming]",[manufacturing],"[farms, agriculture, production]","[miscellaneous, crop, farming]"
3,"[patagonia, chapa, pintura, auto, body, shop, located, comodoro, rivadavia, chubut, province, argentina, company, specializes, providing, auto, body, repair, services]","[automotive, body, repair, services, interior, repair, services]",[services],"[auto, body, shops]","[automotive, body, paint, interior, repair, maintenance]"
4,"[stanica, wodna, pttk, swornegacie, cultural, establishment, located, swornychgaciach, poland, popular, destination, kayakers, tourists, levels, offering, variety, activities, events, establishment, managed, zbigniew, galiåski]","[cultural, activities, accommodation, services, kayak, rentals, small, gastronomy, products, tourism, services, recreational, activities, cultural, center]",[services],"[boat, tours, cruises]","[scenic, sightseeing, transportation, water]"


In [9]:
#lemmatization

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(word) for word in text]

clean['description'] = clean['description'].apply(lemmatize_text)

clean['business_tags'] = clean['business_tags'].apply(lemmatize_text)

clean.head()

,description,business_tags,sector,category,niche
0,"[welchcivils, civil, engineering, construction, company, specializes, designing, building, utility, network, connection, across, uk, offer, multiutility, solution, combine, electricity, gas, water, fibre, optic, installation, single, contract, design, engineer, team, capable, designing, electricity, water, gas, network, existing, network, connection, point, meter, location, development, well, project, management, reinforcement, diversion, provide, custom, connection, solution, take, account, existing, asset, maximize, usage, every, trench, meet, project, deadline, welchcivils, considerable, expertise, installing, gas, electricity, connection, variety, market, category, including, residential, commercial, industrial, project, well]","[construction, service, multiutilities, utility, network, connection, design, construction, water, connection, installation, multiutility, connection, fiber, optic, installation]",[services],"[civil, engineering, services]","[heavy, civil, engineering, construction]"
1,"[kyoto, vegetable, specialist, uekamo, also, known, iwamachi, company, based, kyoto, japan, specializes, sale, vegetable, business, ten, year, offer, collection, vegetable, recipe, keiå, vegetable, recipe, collection, online, shop, company, directly, owned, uekamoo, farm, uekame, farm, lobechi, shijohara, farm, offer, variety, vegetable, product, including, sugukizuke, kamoo, eggplant, also, accept, production, cultivation, according, customer, request, iwaichi, limited, company, us, experience, production, sale, provide, tailored, vegetable, meet, customer, need, also, accepts, cultivation, product, according, requirement]","[wholesale, dualtask, movement, product, cast, iron, product, manufacturer, manufacturing, technology, food, beverage, rice, noodle, highquality, gloss, cast, iron, rice, wholesaler, miscellaneous, crop, farming, health, wellness, product, agricultural, cooperative, medical, practice, based, eastern, medicine, production, rice, pudding]",[manufacturing],"[fruit, vegetable, markets, stores]","[frozen, fruit, juice, vegetable, manufacturing]"
2,"[loidholdhof, integrative, hofgemeinschaft, company, offer, range, service, product, customer, product, handmade, highest, quality, produced, biodynamic, basis, focus, freshness, quality, company, product, range, includes, homemade, bread, honey, beekeeping, syrup, fresh, vegetable, purchased, farm, shop, addition, farm, product, also, farm, shop, cafe, customer, enjoy, fresh, coffee, delicious, cake]","[living, form, farm, cafe, fresh, coffee, community, engagement, freshly, baked, bread, social, interaction, opportunity, fresh, vegetable, homemade, honey, delicious, cake, communityoriented, living, handmade, product, fresh, juice, farm, fresh, product, integrated, farming, community, biodynamic, farming]",[manufacturing],"[farms, agriculture, production]","[miscellaneous, crop, farming]"
3,"[patagonia, chapa, pintura, auto, body, shop, located, comodoro, rivadavia, chubut, province, argentina, company, specializes, providing, auto, body, repair, service]","[automotive, body, repair, service, interior, repair, service]",[services],"[auto, body, shops]","[automotive, body, paint, interior, repair, maintenance]"
4,"[stanica, wodna, pttk, swornegacie, cultural, establishment, located, swornychgaciach, poland, popular, destination, kayakers, tourist, level, offering, variety, activity, event, establishment, managed, zbigniew, galiåski]","[cultural, activity, accommodation, service, kayak, rental, small, gastronomy, product, tourism, service, recreational, activity, cultural, center]",[services],"[boat, tours, cruises]","[scenic, sightseeing, transportation, water]"


In [10]:
clean['description']= clean['description'].apply(lambda x: list(dict.fromkeys(x)) if isinstance(x, list) else x)

clean['business_tags']= clean['business_tags'].apply(lambda x: list(dict.fromkeys(x)) if isinstance(x, list) else x)

# clean = clean.apply(lambda x: sorted(x) if isinstance(x, list) else x)

clean.head()

,description,business_tags,sector,category,niche
0,"[welchcivils, civil, engineering, construction, company, specializes, designing, building, utility, network, connection, across, uk, offer, multiutility, solution, combine, electricity, gas, water, fibre, optic, installation, single, contract, design, engineer, team, capable, existing, point, meter, location, development, well, project, management, reinforcement, diversion, provide, custom, take, account, asset, maximize, usage, every, trench, meet, deadline, considerable, expertise, installing, variety, market, category, including, residential, commercial, industrial]","[construction, service, multiutilities, utility, network, connection, design, water, installation, multiutility, fiber, optic]",[services],"[civil, engineering, services]","[heavy, civil, engineering, construction]"
1,"[kyoto, vegetable, specialist, uekamo, also, known, iwamachi, company, based, japan, specializes, sale, business, ten, year, offer, collection, recipe, keiå, online, shop, directly, owned, uekamoo, farm, uekame, lobechi, shijohara, variety, product, including, sugukizuke, kamoo, eggplant, accept, production, cultivation, according, customer, request, iwaichi, limited, us, experience, provide, tailored, meet, need, accepts, requirement]","[wholesale, dualtask, movement, product, cast, iron, manufacturer, manufacturing, technology, food, beverage, rice, noodle, highquality, gloss, wholesaler, miscellaneous, crop, farming, health, wellness, agricultural, cooperative, medical, practice, based, eastern, medicine, production, pudding]",[manufacturing],"[fruit, vegetable, markets, stores]","[frozen, fruit, juice, vegetable, manufacturing]"
2,"[loidholdhof, integrative, hofgemeinschaft, company, offer, range, service, product, customer, handmade, highest, quality, produced, biodynamic, basis, focus, freshness, includes, homemade, bread, honey, beekeeping, syrup, fresh, vegetable, purchased, farm, shop, addition, also, cafe, enjoy, coffee, delicious, cake]","[living, form, farm, cafe, fresh, coffee, community, engagement, freshly, baked, bread, social, interaction, opportunity, vegetable, homemade, honey, delicious, cake, communityoriented, handmade, product, juice, integrated, farming, biodynamic]",[manufacturing],"[farms, agriculture, production]","[miscellaneous, crop, farming]"
3,"[patagonia, chapa, pintura, auto, body, shop, located, comodoro, rivadavia, chubut, province, argentina, company, specializes, providing, repair, service]","[automotive, body, repair, service, interior]",[services],"[auto, body, shops]","[automotive, body, paint, interior, repair, maintenance]"
4,"[stanica, wodna, pttk, swornegacie, cultural, establishment, located, swornychgaciach, poland, popular, destination, kayakers, tourist, level, offering, variety, activity, event, managed, zbigniew, galiåski]","[cultural, activity, accommodation, service, kayak, rental, small, gastronomy, product, tourism, recreational, center]",[services],"[boat, tours, cruises]","[scenic, sightseeing, transportation, water]"


In [11]:
countries = [country['name'].lower() for country in geonamescache.GeonamesCache().get_countries().values()]
countries.append('uk') 
cities = [city['name'].lower() for city in geonamescache.GeonamesCache().get_cities().values()]

def remove_cities(text):
    if isinstance(text, list):
        # words = text.split()
        # return ' '.join([word for word in words if word not in cities])
        return [word for word in text if word not in cities]
    return text

def remove_countries(text):
    if isinstance(text, list):
        # words = text.split()
        # return ' '.join([word for word in words if word not in countries])
        return [word for word in text if word not in countries]
    return text

clean['description']= clean['description'].apply(remove_cities).apply(remove_countries)

clean.head()

,description,business_tags,sector,category,niche
0,"[welchcivils, civil, engineering, construction, company, specializes, designing, building, utility, network, connection, across, offer, multiutility, solution, combine, electricity, gas, water, fibre, optic, installation, single, contract, design, engineer, team, capable, existing, point, meter, location, development, well, project, management, reinforcement, diversion, provide, custom, take, account, asset, maximize, usage, every, trench, meet, deadline, considerable, expertise, installing, variety, market, category, including, residential, commercial, industrial]","[construction, service, multiutilities, utility, network, connection, design, water, installation, multiutility, fiber, optic]",[services],"[civil, engineering, services]","[heavy, civil, engineering, construction]"
1,"[vegetable, specialist, uekamo, also, known, iwamachi, company, based, specializes, business, ten, year, offer, collection, recipe, keiå, online, shop, directly, owned, uekamoo, farm, uekame, lobechi, shijohara, variety, product, including, sugukizuke, kamoo, eggplant, accept, production, cultivation, according, customer, request, iwaichi, limited, us, experience, provide, tailored, meet, need, accepts, requirement]","[wholesale, dualtask, movement, product, cast, iron, manufacturer, manufacturing, technology, food, beverage, rice, noodle, highquality, gloss, wholesaler, miscellaneous, crop, farming, health, wellness, agricultural, cooperative, medical, practice, based, eastern, medicine, production, pudding]",[manufacturing],"[fruit, vegetable, markets, stores]","[frozen, fruit, juice, vegetable, manufacturing]"
2,"[loidholdhof, integrative, hofgemeinschaft, company, offer, range, service, product, customer, handmade, highest, quality, produced, biodynamic, basis, focus, freshness, includes, homemade, bread, honey, beekeeping, syrup, fresh, vegetable, purchased, farm, shop, addition, also, cafe, enjoy, coffee, delicious, cake]","[living, form, farm, cafe, fresh, coffee, community, engagement, freshly, baked, bread, social, interaction, opportunity, vegetable, homemade, honey, delicious, cake, communityoriented, handmade, product, juice, integrated, farming, biodynamic]",[manufacturing],"[farms, agriculture, production]","[miscellaneous, crop, farming]"
3,"[patagonia, chapa, pintura, auto, body, shop, located, comodoro, rivadavia, chubut, province, company, specializes, providing, repair, service]","[automotive, body, repair, service, interior]",[services],"[auto, body, shops]","[automotive, body, paint, interior, repair, maintenance]"
4,"[stanica, wodna, pttk, swornegacie, cultural, establishment, located, swornychgaciach, popular, destination, kayakers, tourist, level, offering, variety, activity, event, managed, zbigniew, galiåski]","[cultural, activity, accommodation, service, kayak, rental, small, gastronomy, product, tourism, recreational, center]",[services],"[boat, tours, cruises]","[scenic, sightseeing, transportation, water]"


In [12]:
def is_english_word(word):
    return wordnet.synsets(word)

def remove_non_english_words(tokens):
    return [word for word in tokens if is_english_word(word)]

clean['description'] = clean['description'].apply(remove_non_english_words)

clean.head()


,description,business_tags,sector,category,niche
0,"[civil, engineering, construction, company, specializes, designing, building, utility, network, connection, across, offer, solution, combine, electricity, gas, water, fibre, optic, installation, single, contract, design, engineer, team, capable, existing, point, meter, location, development, well, project, management, reinforcement, diversion, provide, custom, take, account, asset, maximize, usage, every, trench, meet, deadline, considerable, expertise, installing, variety, market, category, including, residential, commercial, industrial]","[construction, service, multiutilities, utility, network, connection, design, water, installation, multiutility, fiber, optic]",[services],"[civil, engineering, services]","[heavy, civil, engineering, construction]"
1,"[vegetable, specialist, also, known, company, based, specializes, business, ten, year, offer, collection, recipe, online, shop, directly, owned, farm, variety, product, including, eggplant, accept, production, cultivation, according, customer, request, limited, us, experience, provide, tailored, meet, need, accepts, requirement]","[wholesale, dualtask, movement, product, cast, iron, manufacturer, manufacturing, technology, food, beverage, rice, noodle, highquality, gloss, wholesaler, miscellaneous, crop, farming, health, wellness, agricultural, cooperative, medical, practice, based, eastern, medicine, production, pudding]",[manufacturing],"[fruit, vegetable, markets, stores]","[frozen, fruit, juice, vegetable, manufacturing]"
2,"[integrative, company, offer, range, service, product, customer, handmade, highest, quality, produced, basis, focus, freshness, includes, homemade, bread, honey, beekeeping, syrup, fresh, vegetable, purchased, farm, shop, addition, also, cafe, enjoy, coffee, delicious, cake]","[living, form, farm, cafe, fresh, coffee, community, engagement, freshly, baked, bread, social, interaction, opportunity, vegetable, homemade, honey, delicious, cake, communityoriented, handmade, product, juice, integrated, farming, biodynamic]",[manufacturing],"[farms, agriculture, production]","[miscellaneous, crop, farming]"
3,"[patagonia, auto, body, shop, located, province, company, specializes, providing, repair, service]","[automotive, body, repair, service, interior]",[services],"[auto, body, shops]","[automotive, body, paint, interior, repair, maintenance]"
4,"[cultural, establishment, located, popular, destination, tourist, level, offering, variety, activity, event, managed]","[cultural, activity, accommodation, service, kayak, rental, small, gastronomy, product, tourism, recreational, center]",[services],"[boat, tours, cruises]","[scenic, sightseeing, transportation, water]"


In [13]:
def remove_numbers(text):
    return [word for word in text if not word.isdigit()]

clean['description'] = clean['description'].apply(remove_numbers)

clean.head()

,description,business_tags,sector,category,niche
0,"[civil, engineering, construction, company, specializes, designing, building, utility, network, connection, across, offer, solution, combine, electricity, gas, water, fibre, optic, installation, single, contract, design, engineer, team, capable, existing, point, meter, location, development, well, project, management, reinforcement, diversion, provide, custom, take, account, asset, maximize, usage, every, trench, meet, deadline, considerable, expertise, installing, variety, market, category, including, residential, commercial, industrial]","[construction, service, multiutilities, utility, network, connection, design, water, installation, multiutility, fiber, optic]",[services],"[civil, engineering, services]","[heavy, civil, engineering, construction]"
1,"[vegetable, specialist, also, known, company, based, specializes, business, ten, year, offer, collection, recipe, online, shop, directly, owned, farm, variety, product, including, eggplant, accept, production, cultivation, according, customer, request, limited, us, experience, provide, tailored, meet, need, accepts, requirement]","[wholesale, dualtask, movement, product, cast, iron, manufacturer, manufacturing, technology, food, beverage, rice, noodle, highquality, gloss, wholesaler, miscellaneous, crop, farming, health, wellness, agricultural, cooperative, medical, practice, based, eastern, medicine, production, pudding]",[manufacturing],"[fruit, vegetable, markets, stores]","[frozen, fruit, juice, vegetable, manufacturing]"
2,"[integrative, company, offer, range, service, product, customer, handmade, highest, quality, produced, basis, focus, freshness, includes, homemade, bread, honey, beekeeping, syrup, fresh, vegetable, purchased, farm, shop, addition, also, cafe, enjoy, coffee, delicious, cake]","[living, form, farm, cafe, fresh, coffee, community, engagement, freshly, baked, bread, social, interaction, opportunity, vegetable, homemade, honey, delicious, cake, communityoriented, handmade, product, juice, integrated, farming, biodynamic]",[manufacturing],"[farms, agriculture, production]","[miscellaneous, crop, farming]"
3,"[patagonia, auto, body, shop, located, province, company, specializes, providing, repair, service]","[automotive, body, repair, service, interior]",[services],"[auto, body, shops]","[automotive, body, paint, interior, repair, maintenance]"
4,"[cultural, establishment, located, popular, destination, tourist, level, offering, variety, activity, event, managed]","[cultural, activity, accommodation, service, kayak, rental, small, gastronomy, product, tourism, recreational, center]",[services],"[boat, tours, cruises]","[scenic, sightseeing, transportation, water]"


In [14]:
# def remove_most_common_words(text, n):
#     if isinstance(text, list):
#         sorted_words = sorted(text, key=lambda word: -counter[word])
#         removed_words = sorted_words[:n]
#         print(f"Removed words: {removed_words}")
#         return sorted_words[n:]
#     return text

# # def remove_least_common_words(text, n):
# #     if isinstance(text, list):
# #         sorted_words = sorted(text, key=lambda word: counter[word])
# #         removed_words = sorted_words[:n]
# #         print(f"Removed words: {removed_words}")
# #         return sorted_words[n:]
# #     return text

# def calculate_n_to_remove(text, n):
#     length = len(text)
#     return length // n

# clean['description'] = clean['description'].apply(lambda x: remove_most_common_words(x, calculate_n_to_remove(x, 5)))

# # clean = clean.apply(lambda x: remove_least_common_words(x, calculate_n_to_remove(x, 15)))

# clean.head()


In [15]:
clean.to_csv("../CSV/preped_data.csv", index=False)


In [16]:
# def tokenization(text):
#     if not isinstance(text, str):
#         text = str(text)
#     tokens = re.findall(r'\w+', text)
#     return tokens

# tokens = clean.apply(lambda x: tokenization(x))
# #tokens.head()

In [17]:
# def remove_most_common_words(text, n):
#     return [word for word in text if word not in n]

# all_words = [word for sublist in clean for word in sublist]

# counter = Counter(all_words)
# number = [word for word, _ in counter.most_common(30)]

# print("Most common words and their counts:")
# for word, count in counter.most_common(30):
#     print(f"{word}: {count}")

# clean = clean.apply(lambda x: remove_most_common_words(x, number))

# clean.to_csv("preped_data.csv", index=False)


In [18]:
# def remove_first_n_words(text, n):
#     if isinstance(text, list):
#         return text[n:]
#     return text

# clean = clean.apply(lambda x: remove_first_n_words(x, 4))

# clean.head()

In [19]:
#this is taking a little bit more time (some more seconds) than the previous steps cause we split the text and then we transformt it back to a string

# countries = [country['name'].lower() for country in geonamescache.GeonamesCache().get_countries().values()]
# countries.append('uk') 
# cities = [city['name'].lower() for city in geonamescache.GeonamesCache().get_cities().values()]

# def remove_cities(text):
#     if isinstance(text, str):
#         words = text.split()
#         return ' '.join([word for word in words if word not in cities])
#     return text

# def remove_countries(text):
#     if isinstance(text, str):
#         words = text.split()
#         return ' '.join([word for word in words if word not in countries])
#     return text

# clean= clean.apply(remove_cities).apply(remove_countries)

# clean.head()

In [20]:
labels = pd.read_csv("../CSV/insurance_taxonomy - insurance_taxonomy.csv",encoding="ISO-8859-1")

labels.head()

,label
0,Agricultural Equipment Services
1,Soil Nutrient Application Services
2,Pesticide Application Services
3,Ornamental Plant Nurseries
4,Landscaping Services


In [21]:
labels = labels.applymap(lambda x: x.lower() if isinstance(x, str) else x)

labels.head()

C:\Users\anama_1lv\AppData\Local\Temp\ipykernel_15640\2941433906.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  labels = labels.applymap(lambda x: x.lower() if isinstance(x, str) else x)


,label
0,agricultural equipment services
1,soil nutrient application services
2,pesticide application services
3,ornamental plant nurseries
4,landscaping services


In [22]:
labels = labels.apply(lemmatize_text)

labels.head()

,label
0,agricultural equipment services
1,soil nutrient application services
2,pesticide application services
3,ornamental plant nurseries
4,landscaping services


In [23]:
labels['label']= labels['label'].apply(lambda x: x.split() if isinstance(x, str) else x)

labels.head()

,label
0,"[agricultural, equipment, services]"
1,"[soil, nutrient, application, services]"
2,"[pesticide, application, services]"
3,"[ornamental, plant, nurseries]"
4,"[landscaping, services]"


In [24]:
labels.to_csv("../CSV/labels.csv", index=False)